In [1]:
import xarray as xr
import yaml
import zarr
import numpy as np
import chunk_tools  # local package to outsource some basic functions

In [2]:
def to_zarr_store(path, outds, timechunk, order):
    store = zarr.storage.DirectoryStore(
        path, normalize_keys=False, dimension_separator="/"
    )
    outds.to_zarr(
        store,
        encoding=chunk_tools.get_encodings(
            outds=outds, timechunk=timechunk, order=order
        ),
    )  # , compute=False
    store.close()

In [3]:
def process_3d(curr_conf, ds):
    ds = ds.rename(curr_conf["renames"])
    return ds

In [4]:
def convert_files(zoom, subset):
    files = (
        f"/large/sftpgo/data/NICAM/hackathon/2020/05/ms_*_p25.nc"
    )
    outfile = f"{output_dir}/NICAM_{subset}_z{zoom}.zarr"
    curr_conf = config[subset]
    timechunk = curr_conf["chunks"]["time"]

    ds = xr.open_mfdataset(files, chunks=curr_conf["chunks"])
    out_ds = process_3d(curr_conf, ds)

    ! rm -rf {outfile}
    to_zarr_store(path=outfile, outds=out_ds, timechunk=timechunk, order=zoom)
    return outfile

In [5]:
output_dir = "/large/work/florian"
config = yaml.safe_load(open("nicam_to_zarr.yaml"))
for subset in ("3d6h",):
    for zoom in (9, ):
        outfile = convert_files(zoom=zoom, subset=subset)
        ds_new = xr.open_dataset(outfile)

In [6]:
ds_new

<xarray.Dataset> Size: 234GB
Dimensions:  (time: 124, lev: 25, cell: 3145728)
Coordinates:
  * lev      (lev) float64 200B 1e+03 975.0 950.0 925.0 ... 20.0 10.0 5.0 1.0
  * time     (time) datetime64[ns] 992B 2020-05-01T06:00:00 ... 2020-06-01
Dimensions without coordinates: cell
Data variables:
    healpix  int32 4B ...
    hur      (time, lev, cell) float32 39GB ...
    hus      (time, lev, cell) float32 39GB ...
    ta       (time, lev, cell) float32 39GB ...
    ua       (time, lev, cell) float32 39GB ...
    va       (time, lev, cell) float32 39GB ...
    wa       (time, lev, cell) float32 39GB ...
Attributes:
    comment:  Be careful that definition of time coordinate depends on the da...
    history:  Generated by mod_netcdf.f90.
    title:    NICAM data output

In [7]:
!cat {outfile}/.zmetadata

{
    "metadata": {
        ".zattrs": {
            "comment": "Be careful that definition of time coordinate depends on the dataset and time mode (snapshot/average). Data source: ../../../../..//run3389/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3390/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3391/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3392/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3393/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3394/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3395/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3396/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3397/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3398/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3399/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3400/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3401/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3402/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3403/3dz6h/z09/z2pre/ms_rh_p25.nc,../../../../..//run3404/

In [ ]:
ds_new

<xarray.Dataset> Size: 234GB
Dimensions:  (time: 124, lev: 25, cell: 3145728)
Coordinates:
  * lev      (lev) float64 200B 1e+03 975.0 950.0 925.0 ... 20.0 10.0 5.0 1.0
  * time     (time) datetime64[ns] 992B 2020-05-01T06:00:00 ... 2020-06-01
Dimensions without coordinates: cell
Data variables:
    healpix  int32 4B ...
    hur      (time, lev, cell) float32 39GB ...
    hus      (time, lev, cell) float32 39GB ...
    ta       (time, lev, cell) float32 39GB ...
    ua       (time, lev, cell) float32 39GB ...
    va       (time, lev, cell) float32 39GB ...
    wa       (time, lev, cell) float32 39GB ...
Attributes:
    comment:  Be careful that definition of time coordinate depends on the da...
    history:  Generated by mod_netcdf.f90.
    title:    NICAM data output